<a href="https://colab.research.google.com/github/MsAsterisk1/ERM_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: NVIDIA GeForce RTX 2060 (UUID: GPU-100f20af-1b36-5adc-0795-f06b020c050b)


In [3]:
# !git clone https://github.com/MsAsterisk1/ERM_GDRO.git
# %cd /content/ERM_GDRO

In [4]:
# !git pull

In [5]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
from utils.process_data_utils import  get_CivilComments_DataLoaders, get_CivilComments_Datasets, get_CivilComments_df
from loss import ERMLoss, GDROLoss
from train_eval import train, evaluate, train_epochs
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from torch import nn

Now lets try getting Bert working

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [6]:
df = get_CivilComments_df()

In [ ]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [ ]:
train,cv,test=  get_CivilComments_DataLoaders(datasets=datasets)

This is how we would initialize model

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, classes=2, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, classes)
        self.relu = nn.ReLU()

    def forward(self, X):
        input_id, mask = X
        pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)[0][:,0]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
model = BertClassifier().to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
# loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, train, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


In [ ]:
torch.save(model.state_dict(), './weights_ERM')


In [ ]:
# from google.colab import files
# files.download('./weights_ERM')

In [ ]:
model = BertClassifier().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.01)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = GDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 8, vector_subclass=True)

train_epochs(epochs, train, test, model, loss_fn, optimizer, vector_subclass=True, verbose=True, num_subclasses=8)


In [ ]:
torch.save(model.state_dict(), './weights_GDRO')
